# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
%pip install textdistance

In [2]:
from collections import Counter
import re

corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

In [3]:
import textdistance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances


def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    return closest[0]

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [4]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

get_closest_hybrid_match('солне', X, vec)

('солнце', 0.8333333333333334)

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [5]:
corpus_symspell = open('wiki_data.txt', encoding='utf8').read()
vocab_symspell = Counter(re.findall('\w+', corpus_symspell.lower()))

In [6]:
def edition(word):
    "Создаем кандидатов, в которых удалена одна буква"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def collect_dict_symspell(word):
    dict_symspell = {}
    for var in edition(word):
        dict_symspell[var] = word
    return dict_symspell

In [7]:
# собираем словарь всех возможных удалений
dict_symspell = {}
for word in vocab:
    for var in collect_dict_symspell(word):
        dict_symspell[var] = word

In [17]:
# word_check = 'опофеоз'
# variants = edition(word_check)
# print(variants)

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def get_correction(word):
    variants = edition(word)
    corrections = {}

    for variant in variants:
        if variant in dict_symspell:
            corrections[dict_symspell[variant]] = get_closest_hybrid_match(variant, X, vec)[1]

    corr = {k: v for k, v in reversed(sorted(corrections.items(), key=lambda item: item[1]))}

    max_probability = 0
    # for key in list(corr.keys()):
    #     if corr[key] < max_probability:
    #         break
    #     max_probability = corr[key]
    if list(corr.keys()):
      return list(corr.keys())[0]
    else:
      return [None]

In [9]:
from tqdm.notebook import tqdm
from string import punctuation

punctuation += "«»—…“”"
punct = set(punctuation)

In [24]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [25]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше

        predicted = cashed.get(pair[1], get_correction(pair[1]))
        cashed[pair[1]] = predicted


        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)